# Setup

In [1]:
# As usual, a bit of setup
import time
from time import process_time
import numpy as np
import matplotlib.pyplot as plt
import LOUPE.WILLOW.loupe as lp
import tensorflow as tf
import h5py
import pandas as pd
import csv
import copy
import math
from utils.data_utils import *
import sys
import re
from utils.spj import Config
from utils.spj import SPJ
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

home_dir = "/home/martnzjulio_a/songze"
#home_dir = "/home/songzeli"

/home/shared/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:
Use the retry module or similar alternatives.


# Training Function

In [2]:
def model(all_train, all_val, learning_rate, num_epochs, minibatch_size, num_steps, print_cost = True):
    """
    Implements a tensorflow neural network: C3D->ATTENTION->CAPTIONING
    
    Arguments:
    H_train -- training set, of shape = [n_train,num_c3d_features,num_proposals]
    Y_train -- caption labels, of shape = [n_train,num_proposals,num_steps+1]
    H_test -- training set, of shape = [n_test,num_c3d_features,num_proposals]
    Y_test -- caption labels, of shape = [n_test,num_proposals,num_steps+1]
    learning_rate -- learning rate of the optimization
    num_epochs -- number of epochs of the optimization loop
    minibatch_size -- size of a minibatch
    print_cost -- True to print the cost every 100 epochs
    
    Returns:
    parameters -- parameters learnt by the model. They can then be used to predict.
    """
    (VideoIds_train, Framestamps_train, H_train, Ipast_train, Ifuture_train, Ycaptions_train, Xcaptions_train) = all_train
    (VideoIds_val, Framestamps_val, H_val,   Ipast_val,   Ifuture_val,   Ycaptions_val,   Xcaptions_val)   = all_val
    
    # Directory to Save Checkpoint
    checkpoint_dir = home_dir + "/checkpoints/"
    tensorboard_dir =  home_dir + "/tensorboard/"
    
    # Reset Graph
    tf.reset_default_graph()    
    
    # For Consistency
    tf.set_random_seed(1)                             
    seed = 3                                         
    
    # Number of Training Examples
    num_train = H_train.shape[0] 
    num_val = H_val.shape[0] 
    
    # to keep track of costs
    costs = []
    
    # Global Epoch Number
    global_step = tf.Variable(0, name='global_step', trainable=False)
    
    # Model
    config = Config()
    spj = SPJ(config)
        
    # Backpropagation: Define the tensorflow optimizer. Use an AdamOptimizer.
    optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(spj.loss, global_step=global_step)
    
    # Initialize all the variables
    init = tf.global_variables_initializer()
    
    # Add ops to save and restore all the variables.
    saver = tf.train.Saver(max_to_keep=10)
    
    # Tensorboard Loss
    training_summary = tf.summary.scalar("training_loss", spj.loss)
    validation_summary = tf.summary.scalar("validation_loss", spj.loss)
    

    # Start the session to compute the tensorflow graph
    with tf.Session() as sess:
    
        # check for latest checkpoint
        latest_checkpoint = tf.train.latest_checkpoint(checkpoint_dir)
        if latest_checkpoint == None:
            # If no check point run the initialization
            print()
            print("No checkpoint exists, initializing parameters...")
            sess.run(init)
        else:
            print()
            print("Restoring from latest checkpoint...")
            saver.restore(sess, latest_checkpoint)
        
        #Tensorboard
        summary_writer = tf.summary.FileWriter(tensorboard_dir,sess.graph)
        
        # Training Loop
        print()
        start = process_time() # Record start time
        for epoch in range(num_epochs):
            
            # Variable to store cost
            epoch_train_loss = 0.0
            epoch_val_loss = 0.0
            
            # Get minibatches
            num_train_minibatches = num_train // minibatch_size 
            num_val_minibatches = num_val // minibatch_size 
            seed = seed + 1
            train_minibatches = random_mini_batches(VideoIds_train, Framestamps_train, H_train, Ipast_train, Ifuture_train, Ycaptions_train, Xcaptions_train, minibatch_size, seed)
            val_minibatches = random_mini_batches(VideoIds_val, Framestamps_val, H_val, Ipast_val,   Ifuture_val,   Ycaptions_val,   Xcaptions_val,   minibatch_size, seed)

            for counter, train_minibatch in enumerate(train_minibatches):
                #print(counter)
                
                # Select minibatch
                (minibatch_VideoIds_train, minibatch_Framestamps_train, minibatch_H_train, minibatch_Ipast_train, minibatch_Ifuture_train, minibatch_Ycaptions_train, minibatch_Xcaptions_train) = train_minibatch
                minibatch_Ycaptions_train = id_2_one_hot_void_padding(minibatch_Ycaptions_train, spj.config.num_classes, void_dim=0)
                
                # Run Train Session
                _ , minibatch_train_loss, train_summ = sess.run([optimizer, spj.loss, training_summary], feed_dict={spj._H: minibatch_H_train, spj._Ipast: minibatch_Ipast_train, spj._Ifuture: minibatch_Ifuture_train, spj._x: minibatch_Xcaptions_train, spj._y: minibatch_Ycaptions_train, spj._batch_size: minibatch_H_train.shape[0]}) #
                epoch_train_loss += minibatch_train_loss / num_train_minibatches
                
            for counter, val_minibatch in enumerate(val_minibatches):
                
                # Select minibatch
                (minibatch_VideoIds_val, minibatch_Framestamps_val, minibatch_H_val, minibatch_Ipast_val, minibatch_Ifuture_val, minibatch_Ycaptions_val, minibatch_Xcaptions_val) = val_minibatch
                minibatch_Ycaptions_val = id_2_one_hot_void_padding(minibatch_Ycaptions_val, spj.config.num_classes, void_dim=0)
                
                # Run Validation Session
                minibatch_val_loss, val_summ = sess.run([spj.loss, validation_summary], feed_dict={spj._H: minibatch_H_val, spj._Ipast: minibatch_Ipast_val, spj._Ifuture: minibatch_Ifuture_val, spj._x: minibatch_Xcaptions_val, spj._y: minibatch_Ycaptions_val, spj._batch_size: minibatch_H_val.shape[0]}) #
                epoch_val_loss += minibatch_val_loss / num_val_minibatches
             
            # Print cost
            if print_cost == True:
                global_epoch = tf.train.global_step(sess, global_step)//num_train_minibatches
                print("Epoch: ", global_epoch)
                print("Mini Batch Loss", minibatch_train_loss)
                print ("Training Loss: ", epoch_train_loss)
                print ("Validation Loss: ", epoch_val_loss)
                summary_writer.add_summary(train_summ, global_step=global_epoch) 
                summary_writer.add_summary(val_summ, global_step=global_epoch) 
            
            # Save Model (every 20 epochs)
            if global_epoch % 10 == 0:
                print("Saving Checkpoint for global_step " + str(global_epoch))
                saver.save(sess, checkpoint_dir + 'model', global_step = global_epoch)
        
        # Save and Print Processed Time
        end = process_time() 
        print()
        print("Time Elapased: ", end - start)
        
        return 0

# Load Vocab

In [3]:
# Load Vocabulary
embedding_size =512
pad_len, num_steps = 50, 50
vocabulary,vocab_size = caption_preprocess(home_dir)
emb_matrix,word2id,id2word = get_wordvector(embedding_size,vocab_size,vocabulary)
num_classes = len(word2id)

# Word Embedding Matrix
emb_matrix, word2id, id2word = get_wordvector(embedding_size,vocab_size,vocabulary) #changed by Songze

Total number of words in all captions:  504895
Vocabulary Size (Unique):  13001


# Load All Data

In [4]:
# Load All Data
all_file = home_dir + "/SPJ/train_5000.csv"
all_ids,all_data,all_padded_proposals,all_padded_framestamps = video_preprocess(home_dir, all_file)

# All Captions
all_padded_sentences,all_padded_sentences_2,all_padded_sentences_id = get_padded_sentences_id(pad_len, all_ids, all_data, word2id) 
Ycaptions_all = np.transpose(copy.deepcopy(all_padded_sentences_2),axes=(0,2,1)).astype(np.int32)
Xcaptions_all = np.transpose(copy.deepcopy(all_padded_sentences),axes=(0,2,1)).astype(np.int32)

# All Features 
VideoIds_all = all_ids
Framestamps_all = all_padded_framestamps
H_all = all_padded_proposals.astype(np.float32)
Ipast_all = temporal_indicator(all_padded_framestamps, mode="past").astype(np.float32)
Ifuture_all = temporal_indicator(all_padded_framestamps, mode="future").astype(np.float32)

data_size = len(all_ids)
print("Number of Examples in Data Set:", data_size)

Number of Examples in Data Set: 4990


# Data Split

In [5]:
# Train, Val, Test Split
minibatch_size = 32
num_train = 2*minibatch_size
num_val = 1*minibatch_size
num_test = 1*minibatch_size

# Training Split
VideoIds_train = VideoIds_all[:num_train]
Framestamps_train = Framestamps_all[:num_train]
Xcaptions_train = Xcaptions_all[:num_train]
Ycaptions_train = Ycaptions_all[:num_train]
H_train = H_all[:num_train]
Ipast_train = Ipast_all[:num_train]
Ifuture_train = Ifuture_all[:num_train]
print("VideoIds_train.shape: ", VideoIds_train.shape)
print("Framestamps_train.shape: ", Framestamps_train.shape)
print("Xcaptions_train.shape: ", Xcaptions_train.shape)
print("Ycaptions_train.shape: ", Ycaptions_train.shape)
print("H_train.shape: ", H_train.shape)
print("Ipast_train.shape: ", Ipast_train.shape)
print("Ifuture_train.shape: ", Ifuture_train.shape)
print()

# Validation Split
VideoIds_val = VideoIds_all[num_train:num_train+num_val]
Framestamps_val = Framestamps_all[num_train:num_train+num_val]
Xcaptions_val = Xcaptions_all[num_train:num_train+num_val]
Ycaptions_val = Ycaptions_all[num_train:num_train+num_val]
H_val = H_all[num_train:num_train+num_val]
Ipast_val = Ipast_all[num_train:num_train+num_val]
Ifuture_val = Ifuture_all[num_train:num_train+num_val]
print("VideoIds_val.shape: ", VideoIds_val.shape)
print("Framestamps_val.shape: ", Framestamps_val.shape)
print("Xcaptions_val.shape: ", Xcaptions_val.shape)
print("Ycaptions_val.shape: ", Ycaptions_val.shape)
print("H_val.shape: ", H_val.shape)
print("Ipast_val.shape: ", Ipast_val.shape)
print("Ifuture_val.shape: ", Ifuture_val.shape)
print()

# Testing Split
VideoIds_test = VideoIds_all[num_train+num_val:num_train+num_val+num_test]
Framestamps_test = Framestamps_all[num_train+num_val:num_train+num_val+num_test]
Xcaptions_test = Xcaptions_all[num_train+num_val:num_train+num_val+num_test]
Ycaptions_test = Ycaptions_all[num_train+num_val:num_train+num_val+num_test]
H_test = H_all[num_train+num_val:num_train+num_val+num_test]
Ipast_test = Ipast_all[num_train+num_val:num_train+num_val+num_test]
Ifuture_test = Ifuture_all[num_train+num_val:num_train+num_val+num_test]
print("VideoIds_test.shape: ", VideoIds_test.shape)
print("Framestamps_test.shape: ", Framestamps_test.shape)
print("Xcaptions_test.shape: ", Xcaptions_test.shape)
print("Ycaptions_test.shape: ", Ycaptions_test.shape)
print("H_test.shape: ", H_test.shape)
print("Ipast_test.shape: ", Ipast_test.shape)
print("Ifuture_test.shape: ", Ifuture_test.shape)

VideoIds_train.shape:  (64,)
Framestamps_train.shape:  (64, 2, 30)
Xcaptions_train.shape:  (64, 30, 50)
Ycaptions_train.shape:  (64, 30, 51)
H_train.shape:  (64, 500, 30)
Ipast_train.shape:  (64, 30, 30)
Ifuture_train.shape:  (64, 30, 30)

VideoIds_val.shape:  (32,)
Framestamps_val.shape:  (32, 2, 30)
Xcaptions_val.shape:  (32, 30, 50)
Ycaptions_val.shape:  (32, 30, 51)
H_val.shape:  (32, 500, 30)
Ipast_val.shape:  (32, 30, 30)
Ifuture_val.shape:  (32, 30, 30)

VideoIds_test.shape:  (32,)
Framestamps_test.shape:  (32, 2, 30)
Xcaptions_test.shape:  (32, 30, 50)
Ycaptions_test.shape:  (32, 30, 51)
H_test.shape:  (32, 500, 30)
Ipast_test.shape:  (32, 30, 30)
Ifuture_test.shape:  (32, 30, 30)


# Execute Training

In [14]:
# Train Model
learning_rate = 0.001
num_epochs = 100
minibatch_size = 1
all_train = (VideoIds_train, Framestamps_train, H_train, Ipast_train, Ifuture_train, Ycaptions_train[:,:,1:], Xcaptions_train)
all_val =   (VideoIds_val, Framestamps_val, H_val,   Ipast_val,   Ifuture_val,   Ycaptions_val[:,:,1:],   Xcaptions_val)
execute = model(all_train, all_val, learning_rate, num_epochs, minibatch_size, num_steps)

[30, 50, 1024]
[30, 50, 512]

Restoring from latest checkpoint...
INFO:tensorflow:Restoring parameters from /home/martnzjulio_a/songze/checkpoints/model-20

Epoch:  21
Mini Batch Loss 663.70447
Training Loss:  497.7943801879883
Validation Loss:  827.5614070892334
Epoch:  22
Mini Batch Loss 562.2668
Training Loss:  453.1530394554138
Validation Loss:  808.6005582809448
Epoch:  23
Mini Batch Loss 269.19955
Training Loss:  440.18187618255615
Validation Loss:  801.5723762512207
Epoch:  24
Mini Batch Loss 611.33875
Training Loss:  435.099134683609
Validation Loss:  800.5948629379272
Epoch:  25
Mini Batch Loss 319.2769
Training Loss:  433.2610697746277
Validation Loss:  803.0967078208923
Epoch:  26
Mini Batch Loss 382.32324
Training Loss:  432.52131700515747
Validation Loss:  801.9396901130676
Epoch:  27
Mini Batch Loss 672.6156
Training Loss:  432.7867228984833
Validation Loss:  803.7895526885986
Epoch:  28
Mini Batch Loss 404.38358
Training Loss:  433.24466037750244
Validation Loss:  802.69

KeyboardInterrupt: 

# Generate Captions (Sample)

In [ ]:
tf.set_random_seed(1)                             
seed = 3            
from utils.data_utils import *

tf.reset_default_graph()
config = Config()
spj = SPJ(config)
global_step = tf.Variable(0, name='global_step', trainable=False)
checkpoint_dir = "/home/martnzjulio_a/songze/checkpoints/"
latest_checkpoint = tf.train.latest_checkpoint(checkpoint_dir)
saver = tf.train.Saver()
optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(spj.loss, global_step=global_step)
train_minibatches = random_mini_batches(H_train, Ipast_train, Ifuture_train, Ycaptions_train, Xcaptions_train, minibatch_size, seed)
(minibatch_H_train, minibatch_Ipast_train, minibatch_Ifuture_train, minibatch_Ycaptions_train, minibatch_Xcaptions_train) = train_minibatches[0]   
with tf.Session() as sess:   
    saver.restore(sess, latest_checkpoint)
    word_id = spj.caption_generation(sess,minibatch_H_train, minibatch_Ipast_train, minibatch_Ifuture_train, minibatch_Xcaptions_train, minibatch_Ycaptions_train)
print ("word_id: ", word_id)

# Generation Captions (Direct)

In [ ]:
def setup_graph_and_saver(learning_rate):
    tf.reset_default_graph()    
    tf.set_random_seed(1)                             
    seed = 3 
    global_step = tf.Variable(0, name='global_step', trainable=False)
    config = Config()
    spj = SPJ(config)
    optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(spj.loss, global_step=global_step)
    init = tf.global_variables_initializer()
    saver = tf.train.Saver()
    return spj, saver, global_step, optimizer, init, seed

def direct_caption_generation(data, learning_rate, minibatch_size, print_cost = True):

    # Extract Test Data
    (VideoIds, Framestamps, H, Ipast, Ifuture, Ycaptions, Xcaptions) = data
    num_data = H.shape[0]
    
    # Setup Graph
    spj, saver, global_step, optimizer, init, seed = setup_graph_and_saver(learning_rate)
    checkpoint_dir = "/home/martnzjulio_a/songze/checkpoints2/"
    
    # Start Session
    with tf.Session() as sess:

        # Check for Latest Checkpoint
        latest_checkpoint = tf.train.latest_checkpoint(checkpoint_dir)
        print("Restoring from latest checkpoint...")
        saver.restore(sess, latest_checkpoint)
        
        # Get minibatches
        num_minibatches = num_data // minibatch_size  
        seed = seed + 1
        minibatches = random_mini_batches(VideoIds, Framestamps, H, Ipast, Ifuture, Ycaptions, Xcaptions, minibatch_size, seed) 

        # For all batchs
        predictions = np.array([])
        labels = np.array([])
        for counter, minibatch in enumerate(minibatches):
            # Select minibatch
            (minibatch_VideoIds, minibatch_Framestamps, minibatch_H, minibatch_Ipast, minibatch_Ifuture, minibatch_Ycaptions, minibatch_Xcaptions) = minibatch
            # Generate Captions
            print("examples, ", counter*32 , " to ", counter*32+32)
            pred, lab = spj.generate_caption_2(sess, minibatch_H, minibatch_Ipast, minibatch_Ifuture, minibatch_Ycaptions)
            print(pred.shape)
            print(lab.shape)
            if counter == 0:
                predictions = pred
                labels = lab
                ids = minibatch_VideoIds
            else:
                predictions = np.concatenate((predictions,pred),axis=0)
                labels = np.concatenate((labels,lab),axis=0)
                ids = np.concatenate((ids, minibatch_VideoIds),axis=0)
                
    return predictions, labels, ids
data = (VideoIds_train, Framestamps_train, H_train, Ipast_train, Ifuture_train, Ycaptions_train, Xcaptions_train)
predictions, labels, ids = direct_caption_generation(data, learning_rate, minibatch_size)

In [ ]:
print_pred_and_labels(predictions, labels, ids, id2word, example=1, proposal=3)

# Compute Bleu Evaluation

In [ ]:
labels # shape: (num_examples, num_proposals, num_steps + 1)
predictions # shape: (num_examples, num_proposals, num_steps + 1)
import nltk

# for each example
for i in num_examples:
    
    # for each proposal
    for p in  num_proposals:
        
        # if row of pads skip
        if sum(labels[i,p])==0:
            break # skip
        
        reference = []
        hypothesis = []
        # extract reference from labels
        for w in num_words:
            reference.append(labels[i,p,w])
            if labels[i,p,w] == 3:
                break # stop
                
        # extract reference from predictions
        for w in num_words:
            hypothesis.append(predictions[i,p,w])
            if labels[i,p,w] == 3:
                break # stop
        
        # comute bleu score
        weights_at_1 = (1.0)
        weights_at_2 = (0.5, 0.5)
        weights_at_3 = (0.33, 0.33, 0.33)
        weights_at_4 = (0.25, 0.25, 0.25, 0.25)
        bleu_at_1 = nltk.translate.bleu_score.sentence_bleu([reference], hypothesis, weights_at_1)
        bleu_at_2 = nltk.translate.bleu_score.sentence_bleu([reference], hypothesis, weights_at_2)
        bleu_at_3 = nltk.translate.bleu_score.sentence_bleu([reference], hypothesis, weights_at_3)
        bleu_at_4 = nltk.translate.bleu_score.sentence_bleu([reference], hypothesis, weights_at_4)
            
            

# Prediction w/ True Feed

In [15]:
def setup_graph_and_saver(learning_rate):
    tf.reset_default_graph()    
    tf.set_random_seed(1)                             
    seed = 3 
    global_step = tf.Variable(0, name='global_step', trainable=False)
    config = Config()
    spj = SPJ(config)
    optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(spj.loss, global_step=global_step)
    init = tf.global_variables_initializer()
    saver = tf.train.Saver()
    return spj, saver, global_step, optimizer, init, seed

def direct_inference(data, learning_rate, minibatch_size):

    # Extract Test Data
    (VideoIds, Framestamps, H, Ipast, Ifuture, Ycaptions, Xcaptions) = data
    num_data = H.shape[0]
    
    # Setup Graph
    spj, saver, global_step, optimizer, init, seed = setup_graph_and_saver(learning_rate)
    checkpoint_dir = "/home/martnzjulio_a/songze/checkpoints/"
    
    # Start Session
    with tf.Session() as sess:

        # Check for Latest Checkpoint
        latest_checkpoint = tf.train.latest_checkpoint(checkpoint_dir)
        print("Restoring from latest checkpoint...")
        saver.restore(sess, latest_checkpoint)
        
        # Get minibatches
        num_minibatches = num_data // minibatch_size  
        seed = seed + 1
        minibatches = random_mini_batches(VideoIds, Framestamps, H, Ipast, Ifuture, Ycaptions, Xcaptions, minibatch_size, seed) 

        # For all batchs
        for counter, minibatch in enumerate(minibatches):
            
            # Select minibatch
            (minibatch_VideoIds, minibatch_Framestamps, minibatch_H, minibatch_Ipast, minibatch_Ifuture, minibatch_Ycaptions, minibatch_Xcaptions) = minibatch
            minibatch_Ycaptions = id_2_one_hot_void_padding(minibatch_Ycaptions, spj.config.num_classes, void_dim=0)
            
            # Feed
            feed = {spj._H: minibatch_H,
                    spj._Ipast: minibatch_Ipast,
                    spj._Ifuture: minibatch_Ifuture,
                    spj._x: minibatch_Xcaptions,
                    spj._y: minibatch_Ycaptions,
                    spj._batch_size: minibatch_H.shape[0]}
            
            # Run Predictions
            pred, lab = sess.run([spj._predictions, spj._y], feed_dict=feed) 
            lab = np.argmax(lab,axis=3)
            
            # Cache Results
            if counter == 0:
                predictions = pred
                labels = lab
                ids = minibatch_VideoIds
            else:
                predictions = np.concatenate((predictions,pred),axis=0)
                labels = np.concatenate((labels,lab),axis=0)
                ids = np.concatenate((ids, minibatch_VideoIds),axis=0)

    return predictions, labels, ids
data = (VideoIds_train, Framestamps_train, H_train, Ipast_train, Ifuture_train, Ycaptions_train[:,:,1:], Xcaptions_train)
predictions2, labels2, ids2 = direct_inference(data, learning_rate, minibatch_size)

[30, 50, 1024]
[30, 50, 512]
Restoring from latest checkpoint...
INFO:tensorflow:Restoring parameters from /home/martnzjulio_a/songze/checkpoints/model-40


In [17]:
print_pred_and_labels(predictions2, labels2, ids2, id2word, example=0, proposal=0)


VIDEO ID             PREDICTION           LABEL               
--------             -----                -----               
v_G9ZEi6eRhi0        the                  there               
v_G9ZEi6eRhi0        the                  are                 
v_G9ZEi6eRhi0        the                  about               
v_G9ZEi6eRhi0        the                  five                
v_G9ZEi6eRhi0        the                  to                  
v_G9ZEi6eRhi0        the                  six                 
v_G9ZEi6eRhi0        the                  young               
v_G9ZEi6eRhi0        the                  boys                
v_G9ZEi6eRhi0        the                  bouncing            
v_G9ZEi6eRhi0        the                  and                 
v_G9ZEi6eRhi0        the                  jumping             
v_G9ZEi6eRhi0        the                  in                  
v_G9ZEi6eRhi0        the                  an                  
v_G9ZEi6eRhi0        the                  enclosed    

0